# Pseudo-Hamiltonian neural networks for PDEs

In this notebook, we will give an example of how to setup and train a neural network on to learn the Cahn–Hilliard equation using `phlearn`. We will also demonstrate how to add an external force to the system, which can also be learnt by the pseudo-Hamiltonian framework. 

For details, see ["Pseudo-Hamiltonian neural networks for learning partial differential equations"](https://arxiv.org/abs/2304.14374).

In [ ]:
# Uncomment for local install: 
# %pip install -e ../phlearn 

In [ ]:
import numpy as np
import torch
import phlearn.phsystems.pde as phsys
import phlearn.phnns as phnn
import matplotlib.pyplot as plt
from phlearn.utils import create_video
from scipy.sparse import spdiags

ttype = torch.float32
torch.set_default_dtype(ttype)

In [ ]:
make_videos = False

### Learning a Cahn–Hilliard system

In this notebook, we consider the one-dimensional (forced) Cahn–Hilliard equation. It is given by
\begin{equation}
u_t  - (\nu u + \alpha u^3 + \mu u_{xx})_{xx} = f(x,t).
\label{eq:ac} \tag{1}
\end{equation}
If $f(x,t) = 0$, we have the standard, i.e. unforced, Cahn–Hilliard equation.

We have
$$
\begin{align*}
\mathcal{V}[u] &= \frac{1}{2} \int_\mathbb{R} \left(\nu u^2 + \frac{1}{2}\alpha u^4  - \mu u_x^2\right)\, dx.
\end{align*}
$$

Also including the force term, we can write the equation on the form
$$
u_t = \frac{\partial^2}{\partial x^2} \dfrac{\delta \mathcal{V}}{\delta u}[u] + g(x,t),
$$
since
$$
\begin{equation*}
\frac{\delta\mathcal{V}}{\delta u}[u] = \nu u + \alpha u^3 + \mu u_{xx}.
\end{equation*}
$$

#### Set up the system

Below is an example of how to set up a Hamiltonian PDE system using the PseudoHamiltonianPDESystem() class. The below block sets up the differential equation that will be used to generate the data.

In [ ]:
period = 1
spatial_points = 100
x = np.linspace(0, period-period/spatial_points, spatial_points)

def setup_CahnHilliard_system(x=x, nu=-1., alpha= 1., mu=-.001, force=None):
    
    M = x.size
    dx = x[-1]/(M-1)
    e = np.ones(M)
    Dp = 1/dx*spdiags([e,-e,e], np.array([-M+1,0,1]), M, M).toarray() # Forward difference matrix
    D1 = .5/dx*spdiags([e,-e,e,-e], np.array([-M+1,-1,1,M-1]), M, M).toarray() # Central difference matrix
    D2 = 1/dx**2*spdiags([e,e,-2*e,e,e], np.array([-M+1,-1,0,1,M-1]), M, M).toarray() # 2nd order central difference matrix

    def dissintegral(u):
        return 1/2*np.sum(nu*u**2 + 1/2*alpha*u**4 - mu*(np.matmul(Dp,u.T)**2).T, axis=1)

    def dissintegral_grad(u):
        return nu*u + alpha*u**3 + mu*u@D2
    
    def initial_condition():
        M = x.size
        P = (x[-1]-x[0])*M/(M-1)
        def sampler(rng):
            a1, a2 = rng.uniform(0.01, .05, 2)
            a3, a4 = rng.uniform(0.01, .2, 2)
            k1, k2, k3, k4 = rng.integers(1, 6, 4)
            u0 = 0
            u0 += a1*np.cos(2*k1*np.pi/P*x)
            u0 += a2*np.cos(2*k2*np.pi/P*x)
            u0 += a3*np.sin(2*k3*np.pi/P*x)
            u0 += a4*np.sin(2*k4*np.pi/P*x)
            return u0
        return sampler

    CahnHilliard_system = phsys.PseudoHamiltonianPDESystem(
        nstates=M,
        dissipation_matrix=-D2,
        dissintegral=dissintegral,
        grad_dissintegral=dissintegral_grad,
        external_forces=force,
        init_sampler=initial_condition()
    )

    return CahnHilliard_system

# CahnHilliard_system = setup_CahnHilliard_system()
# It is more efficient to generate data using the implicit midpoint method,
# as in implemented in the phlearn package:
CahnHilliard_system = phsys.CahnHilliardSystem()

#### Generate training data

Use the `CahnHilliard_system` instance to generate training data, which are numerical solutions to the exact PDE.  

In [ ]:
def get_training_data(system, data_points=40, dt=.004, tmax=.004, x=x):
    nt = round(tmax / dt)
    t_axis = np.linspace(0, tmax, nt + 1)
    ntrajectories_train = int(np.ceil(data_points / nt))
    traindata = phnn.generate_dataset(system, ntrajectories_train, t_axis, xspatial=x)
    return traindata, t_axis

traindata, t_axis = get_training_data(CahnHilliard_system)

#### Set up the pseudo-Hamiltonian neural network
We set the kernel sizes of the operators applied to the left-hand side of the PDE, the variational derivative of the Hamiltonian, and the variational derivative of the dissipation integral, respectively.

We will allow additional keyword arguments to be passed to PseudoHamiltonianPDENN() so we have the option of adding a dissipative term and an external force to the model.

In [ ]:
def setup_pseudo_hamiltonian_nn(kernel_sizes, **kwargs):
    dx = x[-1]/(spatial_points-1)
    phmodel = phnn.PseudoHamiltonianPDENN(
        spatial_points,
        kernel_sizes,
        #dissipation_matrix=-1/dx**2*np.array([[[1,-2,1]]]),
        **kwargs,
    )
    return phmodel

kernel_sizes = [1, 0, 3, 0]
phmodel = setup_pseudo_hamiltonian_nn(kernel_sizes,
                                      dissipation_matrix=CahnHilliard_system.dissipation_matrix_flat)

#### Setup a baseline model
To compare against ConservativeDissiaptiveNN() , we will create a baseline model which will approximate the dynamics using a standard fully connected multilayer perceptron. 

In [ ]:
def setup_baseline_nn(**kwargs):
    baseline_nn = phnn.PDEBaselineNN(spatial_points, **kwargs)
    basemodel = phnn.DynamicSystemNN(spatial_points, baseline_nn)
    return basemodel

basemodel = setup_baseline_nn()

#### Train the models

In [ ]:
def train_models(*models, epochs=500, batch_size=32, **kwargs):
    for model in models:
        model, _ = phnn.train(
            model,
            integrator="midpoint",
            traindata=traindata,
            epochs=epochs,
            batch_size=batch_size,
            **kwargs
        )
    return models

phmodel, basemodel = train_models(phmodel, basemodel, epochs=2000)

#### Plot the results

We compare the learned model against the true PDE by integrating from an initial condition not in the training data and on a longer time period thain in the training data.

In [ ]:
def get_solutions(system, phmodel, basemodel, u0, t_axis):
    u_exact, *_ = system.sample_trajectory(t_axis, x0=u0)
    u_phnn, _ = phmodel.simulate_trajectory(integrator=False, t_sample=t_axis, x0=u0, xspatial=x)
    if basemodel is not None:
        u_baseline, _ = basemodel.simulate_trajectory(
            integrator=False, t_sample=t_axis, x0=u0, xspatial=x
        )
    else:
        u_baseline = None
    return u_exact, u_phnn, u_baseline


def plot_solutions(u_exact, u_model, t_axis, model='', y=None):
    N = u_exact.shape[0]
    lw = 2
    colors = [(0,0.4,1),(1,0.7,0.3),(0.2,0.7,0.2),(0.8,0,0.2),(0.5,0.3,.9)]
    if N > 1:
        for i in range(5):
            fig = plt.figure(figsize=(4,2)) 
            plt.plot(x, u_model[int(i*(N-1)/4),:], color = colors[0] if model=='PHNN' else colors[1],
                     linestyle='-', linewidth=lw, label=f't = {i/4*t_axis[-1]}, model')
            plt.plot(x, u_exact[int(i*(N-1)/4),:], color = 'k', linestyle='--', linewidth=1, label=f't = {i/4*t_axis[-1]}, true') 
            plt.xlabel('$x$', fontsize=12)
            plt.ylabel('$u$' if y is None else y, fontsize=12)
            plt.title(model+' model vs. ground truth', fontsize=14)
            plt.legend()
            plt.show()
    else:
        fig = plt.figure(figsize=(7,4))
        plt.plot(x, u_exact[0,:], 'k--', linewidth=lw, label='True')
        plt.plot(x, u_model[0,:], color = colors[4], linestyle='-', linewidth=lw, label='Model')
        plt.xlabel('$x$', fontsize=12)
        plt.ylabel('$u$' if y is None else y, fontsize=12)
        plt.title(model+' model vs. ground truth', fontsize=14)
        plt.legend()
        plt.show()

a1, a2 = 0.01, 0.02
a3, a4 = 0.1, 0.06
k1, k2, k3, k4 = 1, 2, 2, 5
P = (x[-1]-x[0])*x.size/(x.size-1)
u0 = a1*np.cos(2*k1*np.pi/P*x)
u0 += a2*np.cos(2*k2*np.pi/P*x)
u0 += a3*np.sin(2*k3*np.pi/P*x)
u0 += a4*np.sin(2*k4*np.pi/P*x)
t_test = np.linspace(0, .02, 201)

u_exact, u_phnn, u_baseline = get_solutions(CahnHilliard_system, phmodel, basemodel, u0, t_test)
plot_solutions(u_exact, u_phnn, t_test, 'PHNN')
plot_solutions(u_exact, u_baseline, t_test, 'Baseline')

In [ ]:
if make_videos:
    create_video([u_exact, u_phnn, u_baseline], ['Ground truth', 'PHNN', 'Baseline'], x_axis=x,
                 file_name='pure_pm.gif', output_format='GIF')

### Learning a Cahn–Hilliard system with an external force

We now test the PHNN model on a Cahn–Hilliard system with a state- and space-dependent external force acting on the system:
$$
u_t  - (\nu u + \alpha u^3 + \mu u_{xx})_{xx} = f(x,t).
$$

In [ ]:
def F(u, t):
    return np.where((0.3 < x) & (x < 0.7), 30*u, 0)
kernel_sizes = [1, 0, 3, 1]

In [ ]:
#disturbed_CahnHilliard_system = setup_CahnHilliard_system(external_forces=F)
disturbed_CahnHilliard_system = phsys.CahnHilliardSystem(x=x, external_forces=F,
                                                         init_sampler=phsys.initial_condition_ch(x))

In [ ]:
ext_forces_nn = phnn.PDEExternalForcesNN(spatial_points, hidden_dim=100,
                                    timedependent=True, spacedependent=True, statedependent=True,
                                    period=period)
#phmodel = setup_pseudo_hamiltonian_nn(kernel_sizes, external_forces_est=ext_forces_nn)
# We inform the model of the R operator:
phmodel = setup_pseudo_hamiltonian_nn(kernel_sizes,
                                      dissipation_matrix=disturbed_CahnHilliard_system.dissipation_matrix_flat,
                                      external_forces_est=ext_forces_nn)
basemodel = setup_baseline_nn(spacedependent=True, period=period)
traindata, t_axis = get_training_data(disturbed_CahnHilliard_system)
phmodel, basemodel = train_models(phmodel, basemodel, epochs=2000)

In [ ]:
u_exact, u_phnn, u_baseline = get_solutions(disturbed_CahnHilliard_system, phmodel, basemodel, u0, t_test)
plot_solutions(u_exact, u_phnn, t_test, 'PHNN')
plot_solutions(u_exact, u_baseline, t_test, 'Baseline')

In [ ]:
if make_videos:
    create_video([u_exact, u_phnn, u_baseline], ['Ground truth', 'PHNN', 'Baseline'], x_axis=x,
                 file_name='forced_pm.gif', output_format='GIF')

In [ ]:
F_exact = disturbed_CahnHilliard_system.external_forces(u_exact, t_test)#.reshape(1,-1)
F_phnn = phmodel.external_forces(torch.tensor(u_phnn.reshape(-1,1,u_phnn.shape[-1]), dtype=ttype),
                               torch.tensor(t_test.reshape(-1,1,1),dtype=ttype),
                               torch.tensor(np.tile(x.reshape(-1, 1, 1), u_phnn.shape[0]).T, dtype=ttype)
                              ).detach().numpy().reshape(u_phnn.shape)
F_phnn_corrected = 1/phmodel.A().sum().detach().numpy()*F_phnn
plot_solutions(F_exact, F_phnn_corrected, t_test, 'PHNN', 'External force')

#### Removing the forces

Since we learned the external forces by a separate neural network in the PHNN model, we can remove these from the model and compare to the exact system without forces:

In [ ]:
phmodel_no_force = setup_pseudo_hamiltonian_nn(kernel_sizes=[kernel_sizes[0], 0, kernel_sizes[2], 0],
                                               dissipation_matrix=phmodel.R(),
                                               lhs_matrix=phmodel.A(),
                                               dissintegral_true=phmodel.dissintegral,
                                               grad_dissintegral_true=phmodel.dV,
                                              )

u_exact, u_phnn, _ = get_solutions(CahnHilliard_system, phmodel_no_force, None, u0, t_test)
plot_solutions(u_exact, u_phnn, t_test, 'PHNN')

In [ ]:
if make_videos:
    create_video([u_exact, u_phnn], ['Ground truth', 'PHNN'], x_axis=x,
                 file_name='pm.gif', output_format='GIF')